In [1]:
import csv
import numpy as np
import pandas as pd
import pprint  
import collections
from datetime import datetime
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
lm=linear_model.LinearRegression()

In [2]:
segment3_df = pd.DataFrame(pd.read_csv("segment3_processing_time.csv"))
pd.set_option('display.max_columns', 80)
print(segment3_df.shape)
print(len(segment3_df['CASE_ID'].unique()))

(8505, 58)
758


In [3]:
segment3_df.head(3)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,CreateTime,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,REFERRAL_ID,APPLICATION_ID,APPLICATION_LINE_ID,STU_ID,STU_CITIZEN_CD,STU_CITIZEN_NM,STU_BIRTH_DT,STU_ABOR_TSI_CD,STU_ABOR_TSI_NM,STU_BIRTH_CNTRY_CD,STU_BIRTH_CNTRY_NM,STU_CTZN_CNTRY_CD,STU_CTZN_CNTRY_NM,SUBMISSION_METHOD,ENTRY_TYPE,ASSESSMENT_DECISION_SCORE,ASSESSMENT_DECISION_NM,ASSESSMENT_OUTCOME_NM,LAST_COUNTRY_EDUCATION,PREV_INST_NM_MAIN,PREV_INST_NM_ALT1,PREV_INST_NM_ALT2,PREFERENCE_NO,SPK_CD,SPK_FULL_TITLE,ORG_UNIT_CD,AVAIL_ORG_UNIT_NM,SUPPORTING_ORG,STUDY_MODE_CD,RETURNING_DEFERRAL,CURR_AVAIL_YR,CURR_STUDY_PERIOD,CURR_SPRD_CD,CURR_SPRD_NM,ORIG_AVAIL_YR,ORIG_STUDY_PERIOD,ORIG_SPRD_CD,ORIG_SPRD_NM,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,ProcessingTime
0,502030|95557|149167,Assessment Decision - Qualified,Assessment Decision - Qualified,28/7/2016 16:10:40,2016-07-28 16:10:40,THOMPSONN,Faculty,1,Y,2. Perform Assessment,Part 4 - Assessment Decision,NaN,95557,149167,502030,1,Australian,27/9/1991 0:00:00,N,No,1100.0,Australia,1100.0,Australia,Online Application,NaN,0.0,Qualified,Successful,Australia,Monash University,Monash University,NaN,1,MC-ENG,Master of Engineering,400,Engineering,NaN,STD,Y,2019,Start Year Intake,STYI,Start Year Intake,2017,Start Year Intake,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AS,Y,$ACC,N,0
1,502030|95557|149167,Assessment Decision - Qualified,Assessment Decision - Qualified,28/7/2016 16:01:11,2016-07-28 16:01:11,THOMPSONN,Faculty,2,N,2. Perform Assessment,Part 4 - Assessment Decision,NaN,95557,149167,502030,1,Australian,27/9/1991 0:00:00,N,No,1100.0,Australia,1100.0,Australia,Online Application,NaN,0.0,Qualified,Successful,Australia,Monash University,Monash University,NaN,1,MC-ENG,Master of Engineering,400,Engineering,NaN,STD,Y,2019,Start Year Intake,STYI,Start Year Intake,2017,Start Year Intake,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AS,Y,$ACC,N,0 days 00:09:29
2,502030|95557|149167,Manage Acceptance,Manage Acceptance,24/5/2016 13:26:06,2016-05-24 13:26:06,502030,NaN,3,N,4. Post offer,Part 2 - Application Line ID Workflow,NaN,95557,149167,502030,1,Australian,27/9/1991 0:00:00,N,No,1100.0,Australia,1100.0,Australia,Online Application,NaN,0.0,Qualified,Successful,Australia,Monash University,Monash University,NaN,1,MC-ENG,Master of Engineering,400,Engineering,NaN,STD,Y,2019,Start Year Intake,STYI,Start Year Intake,2017,Start Year Intake,STYI,Start Year Intake,MCW,HDCW,Full Degree,AF,AS,Y,$ACC,N,65 days 02:35:05


In [4]:
segment3_info = dict()
for i in range(segment3_df.shape[0]):
    for ids in segment3_df['CASE_ID'].unique():
        if segment3_df['CASE_ID'].iloc[i] == ids:
            if ids not in segment3_info:
                segment3_info[ids] = [segment3_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i]]
            else:
                segment3_info[ids] += [segment3_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i]]                  

#### Workflow Segment 1

In [6]:
ws1_index = dict()
for i in range(segment3_df.shape[0]):
    for ids in segment3_df['CASE_ID'].unique():
        if ids == segment3_df['CASE_ID'].iloc[i]:
            if ids not in ws1_index:
                if segment3_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Send for Assessment':
                    flag2 = i
                    ws1_index[ids] = [flag2]
            else:
                if segment3_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Submit':
                    flag1 = i
                    ws1_index[ids] += [flag1]
                    
print(len(segment3_info))
print(len(ws1_index))

758
647


In [7]:
ws1_list = dict()
for k,v in ws1_index.items():
    if len(v) >= 2:
        Start_index = max(v)
        End_index = min(v)
        records = segment3_df.loc[End_index:Start_index]
        ws1_list[k] = records['WRKFLW_TSK_MAP_GROUPED']

In [9]:
ws1_info = pd.DataFrame(
    [[k,v] for k,l in ws1_list.items() for v in l.tolist()],
    columns = ['CASE_ID','WRKFLW_TSK_MAP_GROUPED']
)
print(len(ws1_info))

4824


In [10]:
ws1_info.head(5)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED
0,502030|95557|149167,Send for Assessment
1,502030|95557|149167,AR - Satisfied - ELR
2,502030|95557|149167,AR - Satisfied - previous or current study
3,502030|95557|149167,AR - Satisfied - advanced standing
4,502030|95557|149167,AR - Satisfied - application details


In [11]:
# A dictionary store key(applicant)-value(workflow names) pairs based on workflow segment1
ws1_dict = dict()
for k,v in ws1_list.items():
    for s in v.tolist():
        if k not in ws1_dict:
            ws1_dict[k] = [s]
        else:
            ws1_dict[k] += [s]

In [13]:
ws1_list_names=list()
for i in range(ws1_info.shape[0]-1, -1, -1):
    if ws1_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i] not in ws1_list_names:
        ws1_list_names.append(ws1_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i])
        
print(len(ws1_list_names))
print(len(ws1_info['WRKFLW_TSK_MAP_GROUPED'].unique()))

61
61


In [14]:
"""
A Dateframe:
    Column：all unique workflow name appeared in workflow segment1 
    Row：all unique applicants in segment3
"""
segment3_ws1_feature = pd.DataFrame(columns=ws1_info['WRKFLW_TSK_MAP_GROUPED'].unique())
print(segment3_ws1_feature.shape)
ws1_feature_cols = segment3_ws1_feature.columns.values.tolist()
print(len(ws1_feature_cols))

(0, 61)
61


In [15]:
segment3_ws1_feature.insert(0,'CASE_ID',[k for k in segment3_info.keys()])

In [16]:
print(segment3_ws1_feature.shape)
segment3_ws1_feature.loc[:,1:62] = 0
segment3_ws1_feature = segment3_ws1_feature.set_index('CASE_ID')

(758, 62)


In [17]:
segment3_ws1_feature.head(3)

,Send for Assessment,AR - Satisfied - ELR,AR - Satisfied - previous or current study,AR - Satisfied - advanced standing,AR - Satisfied - application details,Submit,Release Triage Hold,Put Triage On-Hold,AR - Satisfied - work experience,OC - Created - Other,AR - Not Satisfied - ELR,AR - Satisfied - specialisation,AR - Satisfied - entry points,AR - Satisfied - further information,AR - Satisfied - syllabus,AR - Created - further information,AR - Created - syllabus,AR - Created - work experience,Assessment Decision - Qualified,AR - Satisfied - Other,AR - Created - Other,Assessment Decision - Not Qualified,Assessment Outcome - Not Successful,AR - Satisfied - GAM,Assessment Outcome - Ineligible / Withdrawn,AR - Created - specialisation,AR - Reviewed - further information,AR - Reviewed - advanced standing,Assessment Decision - No Decision,AR - Requested - previous or current study,Back to Triage,AR - Created - previous or current study,AR - Not Satisfied - advanced standing,Release Assessment Hold,Put Assessment On-Hold,AR - Reviewed - work experience,AR - Reviewed - specialisation,AR - Reviewed - entry points,OC - Reviewed - Other,AR - Satisfied - personal statement/expression of interest,Acacdemic - Assessment Decision - Not Qualified,AR - Created - advanced standing,OC - Satisfied - Other,AR - Reviewed - ELR,Release Offer,Create Offer,Assessment Outcome - Successful,Assessment Outcome - No Outcome Set,AR - Requested - further information,AR - Reviewed - application details,AR - Reviewed - GAM,AR - Requested - syllabus,AR - Reviewed - syllabus,AR - Satisfied - agent,AR - Created - personal statement/expression of interest,AR - Not Satisfied - previous or current study,Send Referral,AR - Requested - agent,"AR - Satisfied - financial (sponsorship, scholarship etc)",AR - Created - GAM,AR - Satisfied - guaranteed entry
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
502030|95557|149167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
846123|104356|320614,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
940176|243843|388437,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [79]:
for k in segment3_info.keys():
    for v in segment3_info[k]:
        if v in ws1_feature_cols:
            segment3_ws1_feature.loc[k,v] = 1
        else:
            continue
            
print(segment3_ws1_feature.shape)
print(segment3_ws1_feature['Send for Assessment'].unique())

(758, 61)
[1 0]


In [80]:
segment3_ws1_feature.head(5)

,Send for Assessment,AR - Satisfied - ELR,AR - Satisfied - previous or current study,AR - Satisfied - advanced standing,AR - Satisfied - application details,Submit,Release Triage Hold,Put Triage On-Hold,AR - Satisfied - work experience,OC - Created - Other,AR - Not Satisfied - ELR,AR - Satisfied - specialisation,AR - Satisfied - entry points,AR - Satisfied - further information,AR - Satisfied - syllabus,AR - Created - further information,AR - Created - syllabus,AR - Created - work experience,Assessment Decision - Qualified,AR - Satisfied - Other,AR - Created - Other,Assessment Decision - Not Qualified,Assessment Outcome - Not Successful,AR - Satisfied - GAM,Assessment Outcome - Ineligible / Withdrawn,AR - Created - specialisation,AR - Reviewed - further information,AR - Reviewed - advanced standing,Assessment Decision - No Decision,AR - Requested - previous or current study,Back to Triage,AR - Created - previous or current study,AR - Not Satisfied - advanced standing,Release Assessment Hold,Put Assessment On-Hold,AR - Reviewed - work experience,AR - Reviewed - specialisation,AR - Reviewed - entry points,OC - Reviewed - Other,AR - Satisfied - personal statement/expression of interest,Acacdemic - Assessment Decision - Not Qualified,AR - Created - advanced standing,OC - Satisfied - Other,AR - Reviewed - ELR,Release Offer,Create Offer,Assessment Outcome - Successful,Assessment Outcome - No Outcome Set,AR - Requested - further information,AR - Reviewed - application details,AR - Reviewed - GAM,AR - Requested - syllabus,AR - Reviewed - syllabus,AR - Satisfied - agent,AR - Created - personal statement/expression of interest,AR - Not Satisfied - previous or current study,Send Referral,AR - Requested - agent,"AR - Satisfied - financial (sponsorship, scholarship etc)",AR - Created - GAM,AR - Satisfied - guaranteed entry
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
502030|95557|149167,1,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
846123|104356|320614,1,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
940176|243843|388437,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
758639|264769|421918,1,1,1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
958145|268178|427378,1,0,1,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [94]:
ws1_dict['502030|95557|149167']

['Send for Assessment',
 'AR - Satisfied - ELR',
 'AR - Satisfied - previous or current study',
 'AR - Satisfied - advanced standing',
 'AR - Satisfied - application details',
 'Submit']

#### Workflow Segment2

In [27]:
ws2_index = dict()
for i in range(segment3_df.shape[0]):
    for ids in segment3_df['CASE_ID'].unique():
        if ids == segment3_df['CASE_ID'].iloc[i]:
            if ids not in ws2_index:
                if segment3_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Assessment Outcome - Successful':
                    flag2 = i
                    ws2_index[ids] = [flag2]
                elif segment3_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Assessment Outcome - Not Successful':
                    flag3 = i
                    ws2_index[ids] = [flag3]
            else:
                if segment3_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Send for Assessment':
                    flag1 = i
                    ws2_index[ids] += [flag1]

print(len(segment3_info))
print(len(ws2_index))

758
396


In [29]:
ws2_list = dict()
for k,v in ws2_index.items():
    if len(v) >= 2:
        Start_index = max(v)
        End_index = min(v)
        records = segment3_df.loc[End_index:Start_index]
        ws2_list[k] = records['WRKFLW_TSK_MAP_GROUPED']

In [31]:
ws2_info = pd.DataFrame(
    [[k,v] for k,l in ws2_list.items() for v in l.tolist()],
    columns = ['CASE_ID','WRKFLW_TSK_MAP_GROUPED']
)
print(len(ws2_info))

1776


In [32]:
ws2_info.head(5)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED
0,502030|95557|149167,Assessment Outcome - Successful
1,502030|95557|149167,OC - Created - Other
2,502030|95557|149167,Acacdemic - Assessment Decision - Qualified
3,502030|95557|149167,Assessment Decision - Qualified
4,502030|95557|149167,Send for Assessment


In [33]:
# A dictionary store key(applicant)-value(workflow names) pairs in workflow segment2
ws2_dict = dict()
for k,v in ws2_list.items():
    for s in v.tolist():
        if k not in ws2_dict:
            ws2_dict[k] = [s]
        else:
            ws2_dict[k] += [s]

In [35]:
ws2_list_names=list()
for i in range(ws2_info.shape[0]-1, -1, -1):
    if ws2_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i] not in ws2_list_names:
        ws2_list_names.append(ws2_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i])
        
print(len(ws2_list_names))
print(len(ws2_info['WRKFLW_TSK_MAP_GROUPED'].unique()))

45
45


In [36]:
"""
A Dateframe:
    Column：all unique workflow names appeared in workflow segment2 
    行：all unique applicants in segment3
"""
segment3_ws2_feature = pd.DataFrame(columns=ws2_info['WRKFLW_TSK_MAP_GROUPED'].unique())
print(segment3_ws2_feature.shape)

ws2_feature_cols = segment3_ws2_feature.columns.values.tolist()
print(len(ws2_feature_cols))

(0, 45)
45


In [37]:
segment3_ws2_feature.insert(0,'CASE_ID',[k for k in segment3_info.keys()])

In [38]:
print(segment3_ws2_feature.shape)
segment3_ws2_feature.loc[:,1:46] = 0
segment3_ws2_feature = segment3_ws2_feature.set_index('CASE_ID')

(758, 46)


In [39]:
segment3_ws2_feature.head(3)

,Assessment Outcome - Successful,OC - Created - Other,Acacdemic - Assessment Decision - Qualified,Assessment Decision - Qualified,Send for Assessment,AR - Satisfied - GAM,End My Assessment,Assessment Outcome - Not Successful,Assessment Decision - Not Qualified,Assessment Decision - No Decision,Release Assessment Hold,AR - Requested - further information,AR - Created - further information,Put Assessment On-Hold,Acacdemic - Assessment Decision - Not Qualified,Acacdemic - Assessment Decision - No Decision,AR - Satisfied - Other,AR - Created - Other,AR - Satisfied - previous or current study,AR - Satisfied - syllabus,AR - Satisfied - specialisation,AR - Satisfied - entry points,Assessment Outcome - Higher Preference Successful,Release Triage Hold,Put Triage On-Hold,Back to Triage,AR - Satisfied - ELR,AR - Satisfied - advanced standing,AR - Created - previous or current study,Assessment Decision - System Assessed,AR - Reviewed - ELR,AR - Satisfied - further information,AR - Satisfied - work experience,Restart Offer Workflow,Unsuccessful,Create Offer,AR - Created - specialisation,OC - Satisfied - Other,AR - Not Satisfied - ELR,AR - Reviewed - GAM,Acacdemic - Assessment Outcome - Successful,AR - Satisfied - application details,Provide Response,Send Referral,Close Referral
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
502030|95557|149167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
846123|104356|320614,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
940176|243843|388437,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
for k in segment3_info.keys():
    for v in segment3_info[k]:
        if v in ws2_feature_cols:
            segment3_ws2_feature.loc[k,v] = 1
        else:
            continue

In [81]:
segment3_ws2_feature.head(3)

,Assessment Outcome - Successful,OC - Created - Other,Acacdemic - Assessment Decision - Qualified,Assessment Decision - Qualified,Send for Assessment,AR - Satisfied - GAM,End My Assessment,Assessment Outcome - Not Successful,Assessment Decision - Not Qualified,Assessment Decision - No Decision,Release Assessment Hold,AR - Requested - further information,AR - Created - further information,Put Assessment On-Hold,Acacdemic - Assessment Decision - Not Qualified,Acacdemic - Assessment Decision - No Decision,AR - Satisfied - Other,AR - Created - Other,AR - Satisfied - previous or current study,AR - Satisfied - syllabus,AR - Satisfied - specialisation,AR - Satisfied - entry points,Assessment Outcome - Higher Preference Successful,Release Triage Hold,Put Triage On-Hold,Back to Triage,AR - Satisfied - ELR,AR - Satisfied - advanced standing,AR - Created - previous or current study,Assessment Decision - System Assessed,AR - Reviewed - ELR,AR - Satisfied - further information,AR - Satisfied - work experience,Restart Offer Workflow,Unsuccessful,Create Offer,AR - Created - specialisation,OC - Satisfied - Other,AR - Not Satisfied - ELR,AR - Reviewed - GAM,Acacdemic - Assessment Outcome - Successful,AR - Satisfied - application details,Provide Response,Send Referral,Close Referral
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
502030|95557|149167,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
846123|104356|320614,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0
940176|243843|388437,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
print(segment3_ws2_feature.shape)
print(segment3_ws2_feature['Assessment Outcome - Successful'].unique())
print(segment3_ws2_feature['Assessment Outcome - Not Successful'].unique())

(758, 45)
[1 0]
[0 1]


#### Workflow Segment 3

In [45]:
ws3_index = dict()
for i in range(segment3_df.shape[0]):
    for ids in segment3_df['CASE_ID'].unique():
        if ids == segment3_df['CASE_ID'].iloc[i]:
            if ids not in ws3_index:
                if segment3_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Release Offer':
                    flag2 = i
                    ws3_index[ids] = [flag2]
            else:
                if segment3_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Assessment Outcome - Successful':
                    flag1 = i
                    ws3_index[ids] += [flag1]

print(len(segment3_info))
print(len(ws3_index))

758
253


In [47]:
ws3_list = dict()
for k,v in ws3_index.items():
    if len(v) >= 2:
        Start_index = max(v)
        End_index = min(v)
        records = segment3_df.loc[End_index:Start_index]
        ws3_list[k] = records['WRKFLW_TSK_MAP_GROUPED']

In [49]:
ws3_info = pd.DataFrame(
    [[k,v] for k,l in ws3_list.items() for v in l.tolist()],
    columns = ['CASE_ID','WRKFLW_TSK_MAP_GROUPED']
)
print(len(ws3_info))

1000


In [50]:
ws3_info.head(5)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED
0,502030|95557|149167,Release Offer
1,502030|95557|149167,Create Offer
2,502030|95557|149167,Assessment Outcome - Successful
3,846123|104356|320614,Release Offer
4,846123|104356|320614,Restart Offer Workflow


In [51]:
# A dictionary store key(applicant)-value(workflow names) pairs in workflow segment3
ws3_dict = dict()
for k,v in ws3_list.items():
    for s in v.tolist():
        if k not in ws3_dict:
            ws3_dict[k] = [s]
        else:
            ws3_dict[k] += [s]

In [53]:
ws3_list_names=list()
for i in range(ws3_info.shape[0]-1, -1, -1):
    if ws3_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i] not in ws3_list_names:
        ws3_list_names.append(ws3_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i])
        
print(len(ws3_list_names))
print(len(ws3_info['WRKFLW_TSK_MAP_GROUPED'].unique()))

33
33


In [54]:
"""
A Dateframe:
    Column: all unique workflow names appeared in workflow segment3
    Row：all unique applicants in segment3
"""
segment3_ws3_feature = pd.DataFrame(columns=ws3_info['WRKFLW_TSK_MAP_GROUPED'].unique())
print(segment3_ws3_feature.shape)

ws3_feature_cols = segment3_ws3_feature.columns.values.tolist()
print(len(ws3_feature_cols))

(0, 33)
33


In [55]:
segment3_ws3_feature.insert(0,'CASE_ID',[k for k in segment3_info.keys()])
print(segment3_ws3_feature.shape)

(758, 34)


In [56]:
segment3_ws3_feature.loc[:,1:34] = 0
segment3_ws3_feature = segment3_ws3_feature.set_index('CASE_ID')

In [57]:
segment3_ws3_feature.head(3)

,Release Offer,Create Offer,Assessment Outcome - Successful,Restart Offer Workflow,Allow Workflow Restart,Finalise,Ready to Finalise,Defer,Auto Finalise,Accept,OC - Created - Other,Assessment Decision - Qualified,OC - Satisfied - Other,AR - Satisfied - work experience,Manage Acceptance,Assessment Decision - No Decision,Release Assessment Hold,Put Assessment On-Hold,Decline,AR - Satisfied - previous or current study,Acacdemic - Assessment Outcome - Alternative Offer,Cancel,Send for Assessment,Release Triage Hold,AR - Satisfied - ELR,AR - Satisfied - further information,AR - Satisfied - Other,Unsuccessful,Assessment Outcome - Not Successful,AR - Satisfied - GAM,AR - Created - Other,OC - Created - further information,OC - Requested - Other
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
502030|95557|149167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
846123|104356|320614,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
940176|243843|388437,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
for k in segment3_info.keys():
    for v in segment3_info[k]:
        if v in ws3_feature_cols:
            segment3_ws3_feature.loc[k,v] = 1
        else:
            continue
            
print(segment3_ws3_feature.shape)
print(segment3_ws3_feature['Release Offer'].unique())

(758, 33)
[1 0]


In [82]:
segment3_ws3_feature.head(3)

,Release Offer,Create Offer,Assessment Outcome - Successful,Restart Offer Workflow,Allow Workflow Restart,Finalise,Ready to Finalise,Defer,Auto Finalise,Accept,OC - Created - Other,Assessment Decision - Qualified,OC - Satisfied - Other,AR - Satisfied - work experience,Manage Acceptance,Assessment Decision - No Decision,Release Assessment Hold,Put Assessment On-Hold,Decline,AR - Satisfied - previous or current study,Acacdemic - Assessment Outcome - Alternative Offer,Cancel,Send for Assessment,Release Triage Hold,AR - Satisfied - ELR,AR - Satisfied - further information,AR - Satisfied - Other,Unsuccessful,Assessment Outcome - Not Successful,AR - Satisfied - GAM,AR - Created - Other,OC - Created - further information,OC - Requested - Other
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
502030|95557|149167,1,1,1,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0
846123|104356|320614,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0
940176|243843|388437,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0


#### Workflow Segment 4

In [62]:
ws4_index = dict()
for i in range(segment3_df.shape[0]):
    for ids in segment3_df['CASE_ID'].unique():
        if ids == segment3_df['CASE_ID'].iloc[i]:           
            if ids not in ws4_index:
                if segment3_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Accept':
                    flag2 = i
                    ws4_index[ids] = [flag2]
                elif segment3_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Defer':
                    flag3 = i
                    ws4_index[ids] = [flag3]
            else:
                if segment3_df['WRKFLW_TSK_MAP_GROUPED'].iloc[i] == 'Release Offer':
                    flag1 = i
                    ws4_index[ids] += [flag1]

print(len(segment3_info))
print(len(ws4_index))

758
158


In [63]:
ws4_index

{'502030|95557|149167': [3, 4],
 '846123|104356|320614': [19, 20, 26, 32, 37, 42],
 '758639|264769|421918': [65, 68],
 '958145|268178|427378': [81, 84],
 '982703|311743|493733': [113, 114, 120, 127],
 '1002271|337186|527523': [204, 205],
 '1008291|344923|538678': [258, 261],
 '1007394|346044|540306': [284, 285],
 '982801|348906|544484': [312, 313, 320],
 '1016250|354448|552701': [365, 366, 371],
 '1020768|360055|561313': [442, 443, 448],
 '954468|360490|561961': [500, 505],
 '1022348|362011|564389': [563, 564],
 '1022190|362029|564420': [578, 579],
 '1023330|363366|566519': [605, 606, 611],
 '1023596|363705|567084': [621, 622],
 '1025815|366554|572153': [694, 695],
 '1021374|367597|573577': [748, 749, 754],
 '1027704|369135|575961': [839, 840, 846, 853],
 '1028266|369880|577147': [876, 878],
 '1029502|371827|579839': [953, 954],
 '961605|375543|585341': [1040, 1041],
 '1032525|375569|585384': [1053, 1054],
 '1033537|376798|587229': [1091, 1092],
 '1034446|377904|588943': [1157, 1159],


In [64]:
ws4_list = dict()
for k,v in ws4_index.items():
    if len(v) >= 2:
        Start_index = max(v)
        End_index = min(v)
        records = segment3_df.loc[End_index:Start_index]
        ws4_list[k] = records['WRKFLW_TSK_MAP_GROUPED']

In [65]:
ws4_list

{'502030|95557|149167': 3           Accept
 4    Release Offer
 Name: WRKFLW_TSK_MAP_GROUPED, dtype: object,
 '846123|104356|320614': 19                     Defer
 20             Release Offer
 21    Restart Offer Workflow
 22    Allow Workflow Restart
 23                  Finalise
 24         Ready to Finalise
 25                     Defer
 26             Release Offer
 27    Restart Offer Workflow
 28    Allow Workflow Restart
 29                  Finalise
 30         Ready to Finalise
 31                     Defer
 32             Release Offer
 33    Restart Offer Workflow
 34    Allow Workflow Restart
 35             Auto Finalise
 36                    Accept
 37             Release Offer
 38    Restart Offer Workflow
 39    Allow Workflow Restart
 40             Auto Finalise
 41                    Accept
 42             Release Offer
 Name: WRKFLW_TSK_MAP_GROUPED, dtype: object,
 '758639|264769|421918': 65                     Defer
 66    OC - Satisfied - Other
 67      OC - Cre

In [66]:
ws4_info = pd.DataFrame(
    [[k,v] for k,l in ws4_list.items() for v in l.tolist()],
    columns = ['CASE_ID','WRKFLW_TSK_MAP_GROUPED']
)
print(len(ws4_info))

544


In [67]:
ws4_info.head(5)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED
0,502030|95557|149167,Accept
1,502030|95557|149167,Release Offer
2,846123|104356|320614,Defer
3,846123|104356|320614,Release Offer
4,846123|104356|320614,Restart Offer Workflow


In [68]:
# A dictionary store key(applicant)-value(workflow names) pairs in workflow segment4
ws4_dict = dict()
for k,v in ws4_list.items():
    for s in v.tolist():
        if k not in ws4_dict:
            ws4_dict[k] = [s]
        else:
            ws4_dict[k] += [s]

In [70]:
ws4_list_names=list()
for i in range(ws4_info.shape[0]-1, -1, -1):
    if ws4_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i] not in ws4_list_names:
        ws4_list_names.append(ws1_info['WRKFLW_TSK_MAP_GROUPED'].iloc[i])
        
print(len(ws4_list_names))
print(len(ws4_info['WRKFLW_TSK_MAP_GROUPED'].unique()))

517
25


In [71]:
"""
A Dateframe:
    Column：all unique workflow names appeared in workflow segment4 
    Row：all unique applicants in segment3
"""
segment3_ws4_feature = pd.DataFrame(columns=ws4_info['WRKFLW_TSK_MAP_GROUPED'].unique())
print(segment3_ws4_feature.shape)

ws4_feature_cols = segment3_ws4_feature.columns.values.tolist()
print(len(ws4_feature_cols))

(0, 25)
25


In [72]:
segment3_ws4_feature.insert(0,'CASE_ID',[k for k in segment3_info.keys()])

In [73]:
print(segment3_ws4_feature.shape)
segment3_ws4_feature.loc[:,1:26] = 0
segment3_ws4_feature = segment3_ws4_feature.set_index('CASE_ID')

(758, 26)


In [74]:
segment3_ws4_feature.head(3)

,Accept,Release Offer,Defer,Restart Offer Workflow,Allow Workflow Restart,Finalise,Ready to Finalise,Auto Finalise,OC - Satisfied - Other,OC - Created - Other,Manage Acceptance,AR - Satisfied - previous or current study,Assessment Decision - Qualified,AR - Created - Other,Decline,Back to Perform Assessment,PREPARE TO LOCK,AR - Satisfied - ELR,Send for Assessment,Release Triage Hold,Cancel,AR - Satisfied - GAM,AR - Satisfied - syllabus,Assessment Decision - No Decision,OC - Requested - Other
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,
502030|95557|149167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
846123|104356|320614,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
940176|243843|388437,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [75]:
for k in segment3_info.keys():
    for v in segment3_info[k]:
        if v in ws4_feature_cols:
            segment3_ws4_feature.loc[k,v] = 1
        else:
            continue
            
print(segment3_ws4_feature.shape)
print(segment3_ws4_feature['Accept'].unique())
print(segment3_ws4_feature['Defer'].unique())

(758, 25)
[1 0]
[0 1]


In [84]:
segment3_ws4_feature.head(3)

,Accept,Release Offer,Defer,Restart Offer Workflow,Allow Workflow Restart,Finalise,Ready to Finalise,Auto Finalise,OC - Satisfied - Other,OC - Created - Other,Manage Acceptance,AR - Satisfied - previous or current study,Assessment Decision - Qualified,AR - Created - Other,Decline,Back to Perform Assessment,PREPARE TO LOCK,AR - Satisfied - ELR,Send for Assessment,Release Triage Hold,Cancel,AR - Satisfied - GAM,AR - Satisfied - syllabus,Assessment Decision - No Decision,OC - Requested - Other
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,
502030|95557|149167,1,1,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1,1,0,0,0,0,0,0
846123|104356|320614,1,1,1,1,1,1,1,1,0,0,0,1,1,0,0,0,0,1,1,1,0,1,0,0,0
940176|243843|388437,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0
